# co-visitation matrix_v4
各sessionのtailのみ使用 
重複削除

In [1]:
import os
import sys
import gc
import subprocess
from dotenv import load_dotenv
load_dotenv
sys.path.append(os.getenv('UTILS_PATH'))
from tqdm import tqdm
import multiprocessing
import random
from collections import defaultdict

import pandas as pd
import cudf

In [2]:
SEED = 42
random.seed(SEED)

In [3]:
INPUT_DIR = os.getenv('INPUT_DIR')
OUTPUT_DIR = os.getenv('OUTPUT_DIR')
PREP_DIR = os.getenv("PREP_DIR")

In [4]:
CHUNK_N = 400

In [5]:
# データ読み込み
train_sessions = pd.read_pickle(PREP_DIR + "train_sessions.pkl")
test_sessions = pd.read_pickle(PREP_DIR + "test_sessions.pkl")

train_sessions = train_sessions.drop(columns=["type"])
test_sessions = test_sessions.drop(columns=["type"])

train_sessions["ts"] = (train_sessions["ts"]/1000).astype("int32")
test_sessions["ts"] = (test_sessions["ts"]/1000).astype("int32")

sessions = pd.concat([train_sessions, test_sessions])

# chunkのグループ分け
sessions["chunk"] = sessions["session"] % CHUNK_N

del train_sessions, test_sessions
gc.collect()

0

In [6]:
dfs_outer = []
dfs_inner = []

pair_df = pd.DataFrame(columns=["aid_x", "aid_y", "cnt"])
for i, chunk_df in tqdm(sessions.groupby("chunk"), total=CHUNK_N):
    df = cudf.from_pandas(chunk_df)
    df = df.reset_index(drop=True)
    df = df.sort_values(["session", "ts"], ascending=[True, False])
    df = df.drop_duplicates(["session", "aid"], keep="last")
    df["n"] = df.groupby("session").cumcount()
    df = df.loc[df.n<30].drop("n", axis=1)
    df = df.merge(df, on="session")
    df = df.loc[((df.ts_x - df.ts_y).abs() < 24 * 60 * 60) & (df.aid_x != df.aid_y)]
    df = df.groupby(["aid_x", "aid_y"])["session"].count().reset_index()
    df = df.rename(columns = {"session": "cnt"})
    df = df[df["cnt"]>2]
    dfs_inner.append(df.to_pandas())

    if i % 100 == 99:
        if len(dfs_inner) > 0:
            pair_df_inner = pd.concat(dfs_inner)
            pair_df_inner = pair_df_inner.groupby(["aid_x", "aid_y"])["cnt"].sum().reset_index()
            dfs_outer.append(pair_df_inner)
            dfs_inner = []
            del pair_df_inner
            gc.collect()

if len(dfs_inner) > 0:
    pair_df_inner = pd.concat(dfs_inner)
    pair_df_inner = pair_df_inner.groupby(["aid_x", "aid_y"])["cnt"].sum().reset_index()
    dfs_outer.append(pair_df_inner)
    del pair_df_inner
    gc.collect()

del dfs_inner
gc.collect()

pair_df = pd.concat(dfs_outer)
pair_df = pair_df.groupby(["aid_x", "aid_y"])["cnt"].sum().reset_index()
pair_df = pair_df.sort_values(["aid_x", "cnt"], ascending=(True, False))
pair_df = pair_df.groupby("aid_x").head(20)
pair_df.to_pickle(PREP_DIR + f"co_visitation_matrix_v4.pkl")

100%|██████████| 400/400 [01:04<00:00,  6.24it/s]


In [7]:
pair_df

,aid_x,aid_y,cnt
6,3,1180285,159
0,3,170046,63
11,3,1771163,18
5,3,831065,12
9,3,1717856,6
...,...,...,...
371419,1855500,1211253,3
371420,1855500,1233601,3
371423,1855500,1385509,3
371424,1855500,1834065,3
